# Task 2 - Markov Chain Monte Carlo 

In [8]:
import numpy as np

## 1. Provided functions
By calling the function $\color{Brown} {mimo qam cstll(M,Nt)}$ we get the transmitted MIMO-signal $\color{Brown}{x}$, the respective probability distribution $\color{Brown}{Px}$ and the MIMO-labels. 

In order to explore the provided funtions I set $\color{Brown}{M = 16}$ and $\color{Brown}{Nt = 2}$.


In [9]:
def ask_cstll(M):
    # returns an M-QAM constellation as an
    # array of dimension Mx1
    X = np.arange(-M+1,M,2,dtype=int)
    X = X.reshape((1,X.size))
    X = X/np.sqrt(np.mean(X**2))
    return X

def qam_cstll(M):
    # returns an M-QAM constellation as an
    # array of dimension Mx1
    M_ask = int(np.sqrt(M))
    tmp = ask_cstll(M_ask).reshape((M_ask,1))
   
    tmp = tmp + 1j*tmp.T
    X = tmp.flatten()
    X = X/np.sqrt(np.mean(np.abs(X)**2))
    
    return X

def mimo_qam_cstll(M, Nt):
    # returns a MIMO constellation for Nt antennas
    # and using a M-QAM constellation on all of them
    
    m = int(np.log2(M))
    X = qam_cstll(M)
    Xidx = np.arange(0,M,dtype=int)
    
    labels = qam_labels(M)
    labels_mimo = np.zeros((M**Nt,m*Nt), dtype=int)
    
    # zeros only...
    pX_mimo = np.zeros((1,M**Nt), dtype=float) / (M**Nt) 
    
    arr = np.empty([M for i in range(Nt)] + [Nt], dtype=complex)
    arr1 = np.empty([M for i in range(Nt)] + [Nt], dtype=int)
    for i, a in enumerate(np.ix_(*[X for ii in np.arange(Nt)])):
        arr[...,i] = a
    for i, a in enumerate(np.ix_(*[Xidx for ii in np.arange(Nt)])):
        arr1[...,i] = a 
    X_mimo = arr.reshape(-1, Nt).T
    X_idx_mimo = arr1.reshape(-1, Nt).T
    for k in np.arange(0,M**Nt):
        labels_mimo[k,:] = np.concatenate(labels[X_idx_mimo[:,k],:],0)
    
    X_mimo = X_mimo / np.sqrt(np.mean(np.linalg.norm(X_mimo,axis=0)**2))
    return (X_mimo, pX_mimo, labels_mimo)

def ask_labels(M):
    # returns the Gray labels of an M-ASK constellation
    # as an array of dimensions M x log2(M)
    m = int(np.log2(M))
    labels = np.zeros((M,m), dtype=int)
    for k in range(1,m+1):
        tmp = np.floor(np.mod(np.arange(0,M)/(2**k)+0.5,2))
        labels[:,-k] = tmp

    return labels

def qam_labels(M):
    # returns te Gray labels of an M-QAM constellation
    # as an array of dimensions M x log2(M)
    
    # get the number of bits for a single 1D (ASK) constellation
    M_ask = int(np.sqrt(M))
    m_ask = int(np.log2(M_ask))
    m = int(m_ask*2)

    
    labels_ask = ask_labels(M_ask)
    labels_qam = np.empty((M,m),dtype=int)
    
    k = 0
    for i in range(0,M_ask):
        for j in range(0,M_ask):
            labels_qam[k,:] = np.hstack([labels_ask[i,:], labels_ask[j,:]])
            k = k + 1
            
    return labels_qam

def bicm_cap_mc(B, L):
    """
    Calculates the BICM capacity of an AWGN channel with input B and loglikelihood
    ratios L via Monte Carlo estimation. Both B and L have dimension #samples x m,
    where m denotes the number of bits of each constellation point.
    """
    m = L.shape[1]
    
    R = m - np.sum(np.mean(np.log2(1+np.exp(-L * (1-2*B))),axis=0))
    
    return R

###### Exploring the code
The transmitted signal X is given as a $\color{Brown}{(2,256)}$ matrix with complex entries. The probability distribution $\color{Brown}{Px}$ has the shape $\color{Brown}{(1,256)}$ and is zero for every entry. The labels are given as a $\color{Brown}{(256,8)}$ matrix. 

(I assume that each column of the matrix X is mapped to a bitvector b, e.g. 2 entries of X are mapped to the 2*4 bitvector. )

In [11]:
print("16-QAM Gray-Labels: \n {}".format(qam_labels(16)))

16-QAM Gray-Labels: 
 [[0 0 0 0]
 [0 0 0 1]
 [0 0 1 1]
 [0 0 1 0]
 [0 1 0 0]
 [0 1 0 1]
 [0 1 1 1]
 [0 1 1 0]
 [1 1 0 0]
 [1 1 0 1]
 [1 1 1 1]
 [1 1 1 0]
 [1 0 0 0]
 [1 0 0 1]
 [1 0 1 1]
 [1 0 1 0]]


In [34]:
x_mimo = mimo_qam_cstll(16, 2)[0]
print("X_mimo:  \n {}".format(x_mimo))
print("X_mimo.shape:  \n {}".format(x_mimo.shape))
print("X_mimo[0]:  \n {}".format(x_mimo[0]))
print("X_mimo[0].shape:  \n {}".format(x_mimo[0].shape))

X_mimo:  
 [[-0.67082039-0.67082039j -0.67082039-0.67082039j -0.67082039-0.67082039j
  -0.67082039-0.67082039j -0.67082039-0.67082039j -0.67082039-0.67082039j
  -0.67082039-0.67082039j -0.67082039-0.67082039j -0.67082039-0.67082039j
  -0.67082039-0.67082039j -0.67082039-0.67082039j -0.67082039-0.67082039j
  -0.67082039-0.67082039j -0.67082039-0.67082039j -0.67082039-0.67082039j
  -0.67082039-0.67082039j -0.67082039-0.2236068j  -0.67082039-0.2236068j
  -0.67082039-0.2236068j  -0.67082039-0.2236068j  -0.67082039-0.2236068j
  -0.67082039-0.2236068j  -0.67082039-0.2236068j  -0.67082039-0.2236068j
  -0.67082039-0.2236068j  -0.67082039-0.2236068j  -0.67082039-0.2236068j
  -0.67082039-0.2236068j  -0.67082039-0.2236068j  -0.67082039-0.2236068j
  -0.67082039-0.2236068j  -0.67082039-0.2236068j  -0.67082039+0.2236068j
  -0.67082039+0.2236068j  -0.67082039+0.2236068j  -0.67082039+0.2236068j
  -0.67082039+0.2236068j  -0.67082039+0.2236068j  -0.67082039+0.2236068j
  -0.67082039+0.2236068j  -0.670820

In [29]:
px_mimo = mimo_qam_cstll(16, 2)[1]
print("Px_mimo:  \n {}".format(px_mimo))
print("Px_mimo.shape:  \n {}".format(px_mimo.shape))

Px_mimo:  
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Px_mimo.shape:  
 (1, 256)


In [28]:
labels_mimo = mimo_qam_cstll(16, 2)[2]
print("labels_mimo:  \n {}".format(labels_mimo))
print("labels_mimo.shape:  \n {}".format(labels_mimo.shape))

labels_mimo:  
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 1 1]
 ...
 [1 0 1 ... 0 0 1]
 [1 0 1 ... 0 1 1]
 [1 0 1 ... 0 1 0]]
labels_mimo.shape:  
 (256, 8)


### 1. Define general parameters and system

In [35]:
Nt = 2
Nr = 2

M = 16 #  16-QAM

X = mimo_qam_cstll(M, Nt)[0]

# noise
mu_n = 0
sigma2_n = 0.1
noise = np.random.randn(Nr,1) * np.sqrt(sigma2_n)

# flat fading channel H (Nr x Nt) with iid entries, zero mean and unit variance
H = np.random.randn(Nr,Nt)

# System:
y = np.dot(H,x) + noise


### 3. Implementation of the Gibbs-Sampler:

In [44]:
# LLR_i = ln(P(b_i = 1| y_vector)/P(b_i = -1|y)):
def compute_llr_i(ith_bit):
    loglikeli_ratio = 0
    
    pass
    

n_samples = 100
n_iters = 100
# Algorithm:
# 1. Initialize t = 0 amd generate initial vector b_zero randomly:
b_zero = np.random.rand(n_samples) # initialization
for t in range(n_iters):
    for i in range(n_samples):
        b_i = b[i]
        llr_i = compute_llr_i(b_i) # TODO: write function!!
        
        #delete next line
        llr_i = 2
        
        # generate random number U uniformly between [0,1]:
        u = np.random.uniform(0,1)
        if u < (1/(1+np.exp(-1*llr_i))):
            b_zero[i] = 1
        else:
            b_zero[i] = -1
        
    

